In [1]:
import osmnx as ox, networkx as nx, matplotlib.cm as cm, pandas as pd, numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import geopandas as gpd
import functools
import community

from scipy import sparse
from scipy.sparse import linalg
import time
from shapely.geometry import Point, LineString, Polygon, MultiPolygon, mapping
from math import sqrt
import pandas as pd
from shapely.ops import cascaded_union
pd.set_option('precision', 10)

In [2]:
import image_city_functions as ic

# Computing landmarkness on paths and nodes

In [3]:
#initialise path, names, etc.

city_name = 'London'
folder_ouptut ='Outputs/'+city_name+'/'
epsg = 27700
crs = {'init': 'epsg:27700', 'no_defs': True}

In [4]:
# try reading nodes, paths, landmarks and sight_lines
# At this point, every element has already been extracted

nodes = gpd.read_file(folder_ouptut+city_name+'_experiment_nodes.shp', driver='ESRI Shapefile')
paths = gpd.read_file(folder_ouptut+city_name+'_experiment_paths.shp', driver='ESRI Shapefile')
landmarks = gpd.read_file(folder_ouptut+city_name+'_landmarks.shp', driver='ESRI Shapefile')
sight_lines = gpd.read_file(folder_ouptut+city_name+'_sight_lines.shp', driver='ESRI Shapefile')

In [5]:
nodesAll = gpd.read_file(folder_ouptut+city_name+'_nodes.shp', driver='ESRI Shapefile')

In [6]:
nodes['coordinates'] = nodes[['x', 'y']].apply(tuple, axis=1)
nodesAll['coordinates'] = nodesAll[['x', 'y']].apply(tuple, axis=1)

In [7]:
nodesT = pd.merge(nodes, nodesAll[['nodeID','coordinates']], left_on = 'coordinates', right_on = 'coordinates')

In [8]:
nodesT['nodeID'] = nodesT['nodeID_y'] 
nodesT.drop(['nodeID_x', 'nodeID_y', 'coordinates'], axis = 1, inplace =True)

In [9]:
nodesT.head()

,x,y,weight,Bc,Sc,Rc400,Rc600,Bc400,Bc600,Bc_sc,Sc_sc,Rc400_sc,Rc600_sc,Bc400_sc,Bc600_sc,height,geometry,nodeID
0,531336.0000001253,180096.9999995109,6.0,27035.0,1476.3252335184,643.0,1327.0,812.0,466.0,0.0899565440,0.6738782619,0.1075737265,0.1179430872,0.0555099809,0.0885258359,2,POINT (531336.0000001253 180096.9999995109),2
1,530201.0000001198,181209.9999995350,37.0,93839.0,1544.0025763110,3184.0,7005.0,2258.0,1200.0,0.3122408779,0.8331905585,0.5333445040,0.6374782688,0.1543614985,0.2279635258,2,POINT (530201.0000001198 181209.999999535),3
2,531996.0000001284,181571.9999994949,5.0,0.0,1393.6939741797,284.0,1084.0,0.0,0.0,0.0000000000,0.4793644829,0.0474195710,0.0957086650,0.0000000000,0.0000000000,2,POINT (531996.0000001284 181571.9999994949),5
3,531725.0000001271,180274.9999995017,29.0,8293.0,1469.6319448975,601.0,1360.0,1284.0,553.0,0.0275942156,0.6581222771,0.1005361930,0.1209625766,0.0877768663,0.1050531915,2,POINT (531725.0000001271 180274.9999995017),6
4,529874.0000001183,180764.9999995416,37.0,1799.0,1511.7207881573,3435.0,7592.0,236.0,125.0,0.0059860116,0.7571993095,0.5754021448,0.6911885808,0.0161334427,0.0237462006,2,POINT (529874.0000001183 180764.9999995416),7


In [10]:
nodes = ic.decision_score(nodesT, landmarks)

C:\Users\g_filo01\sciebo\scripts\Image of the City\image_city_functions.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[i+'_sc'] = (df[i]-df[i].min())/(df[i].max()-df[i].min())
C:\Users\g_filo01\sciebo\scripts\Image of the City\image_city_functions.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if (inverse == True): df[i+'_sc'] = 1-(df[i]-df[i].min())/(df[i].max()-df[i].min())
C:\Users\g_filo01\sciebo\scripts\Image of the City\image_city_functions.py:884: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

In [ ]:
nodes = ic.distant_score(nodes, landmarks, sight_lines)

In [ ]:
# rescale to have higher values for lower values and viceversa
col = ['DD', 'DE']

for i in col: ic.scaling_columnDF(nodes, i, inverse = True)

In [ ]:
# assigning scores to edges, based on the best reassuring landmark along each of them

# spatial_index = landmarks.sindex
# paths['reass'] = 0.0

# for row in paths.itertuples():
    
#     g = row[-2] #geometry
#     fil = g.buffer(25)    
    
#     possible_matches_index = list(spatial_index.intersection(fil.bounds))
#     possible_matches = landmarks.iloc[possible_matches_index]
#     precise_matches = possible_matches[possible_matches.intersects(fil)]

#     if (len(precise_matches)==0): continue
    
#     precise_matches = precise_matches.sort_values(by='score_scal', ascending=False).reset_index()
#     score = precise_matches['score_scal'].loc[0]
#     paths.set_value(row[0], 'reass', score)

In [ ]:
# assigning nodes' score to each edge

paths['nS_vis'] = 0.0
paths['nS_dp'] = 0.0

for row in paths.itertuples():
    u = row[2]
    v = row[3]
    nodes_path = nodes[nodes['nodeID'].isin([u,v])]
    paths.set_value(row[0], 'nS_vis', nodes_path['dp'].sum())
    paths.set_value(row[0], 'nS_dp', nodes_path['dist'].sum())

In [ ]:
# scaling raw scores as before
col = ['reass', 'nS_vis', 'nS_dp'] 

for i in col:
    inverse_scaling(paths, i)

# scaling distances
scaling(paths, 'length')

In [ ]:
paths['land'] = paths['reass_sc']+ paths['nS_vis_sc'] + paths['nS_dp_sc']
scaling(paths, 'land')

In [ ]:
paths['dist&land'] = paths['length_sc']+paths['land_sc']
paths.head()

# Computing edge betweenness 
## Primal graph

In [ ]:
nodes = nodes.drop(nodes[['visible_landmarks']],axis=1)
nodes.gdf_name = 'Nodes_list' #for OSMNx
paths.gdf_name = 'Edges_list' #for OSMNx

In [ ]:
G = ox.gdfs_to_graph(nodes, paths)
t = G.nodes()
pos = {}

for l, item in enumerate(t): pos[l] = (t[l]['x'],t[l]['y'])

Ng = nx.Graph() #Empty graph
Ng = Ng.to_undirected()
Ng.add_nodes_from(pos.keys()) #Add nodes preserving coordinates

for i, item in enumerate(Ng.nodes()):
    Ng.node[i]['x']=pos[i][0]
    Ng.node[i]['y']=pos[i][1]

for i, item in enumerate(G.edges()):
    Ng.add_edge(item[0], item[1])
    Ng[item[0]][item[1]]['land_sc']=G[item[0]][item[1]][0]['land_sc']
    Ng[item[0]][item[1]]['dist&land']=G[item[0]][item[1]][0]['dist&land']
    Ng[item[0]][item[1]]['streetID']=G[item[0]][item[1]][0]['streetID']

In [ ]:
Lb = nx.edge_betweenness_centrality(Ng, weight='land_sc', normalized=False) 

In [ ]:
ELb = nx.edge_betweenness_centrality(Ng, weight='dist&land', normalized=False)

In [ ]:
def to_df(list_dict, list_col):
    
    df = pd.DataFrame(list_dict).T
    df.columns = ['d{}'.format(i) for i, col in enumerate(df, 1)]
    df.columns = list_col
    
    return(df)

In [ ]:
edge_id={}
for i, g in Ng.edges(): edge_id[(i,g)]=Ng[i][g]['streetID']

edges_df = to_df([Lb, ELb, edge_id], ["Lb", "ELb","streetID"])
edges_df.streetID = edges_df.streetID.astype(int)

In [ ]:
paths_tmp = pd.merge(paths, edges_df, left_on = 'streetID', right_on = 'streetID', how='left')

In [ ]:
def plotGDF(gdf, column, title):
    f, ax = plt.subplots(1, figsize=(15, 15))
    gdf.plot(ax=ax, column=column, cmap='OrRd', scheme='Fisher_Jenks', linewidth=0.5, legend=True)
    f.suptitle(title)
    plt.axis('equal')
    leg = ax.get_legend()
    leg.set_bbox_to_anchor((0., 0., 0.2, 0.2))
    ax.set_axis_off()

In [ ]:
f, ax = plt.subplots(1, figsize=(10, 10))
nodes.plot(ax=ax, column='dist', cmap='OrRd', scheme='Fisher_Jenks', markersize=5)
plt.axis('equal')
ax.set_axis_off()

In [ ]:
plotGDF(paths_tmp,'Eb', 'Euclidean Betweenness')
plotGDF(paths_tmp,'Lb', 'Landmark Betweenness')
plotGDF(paths_tmp,'ELb', 'Landmark and distance Betweenness')

In [ ]:
paths.head()

## Dual graph

In [ ]:
nodes_dual = gpd.read_file(folder_ouptut+city_name+'_nodesDual.shp', driver='ESRI Shapefile')
edges_dual = gpd.read_file(folder_ouptut+city_name+'_edgesDual.shp', driver='ESRI Shapefile')

In [ ]:
# assigning landmark scores to links between centroids (this is artificial)

edges_dual.u = edges_dual.u.astype(int)
edges_dual.v = edges_dual.v.astype(int)
edges_dual.key = edges_dual.key.astype(int)

for row in edges_dual.itertuples():
    landmarkness_u = paths['land'][paths.streetID==row[1]].loc[row[1]] #row[1]==u
    landmarkness_v = paths['land'][paths.streetID==row[2]].loc[row[2]] #row[2]==v
    landmarkness = landmarkness_u+landmarkness_v
    edges_dual.set_value(row[0], 'land', landmarkness)

In [ ]:
scaling(edges_dual, 'rad')

In [ ]:
# scaling raw scores and deviation

scaling(edges_dual, 'land')  
edges_dual['angle&land'] = edges_dual['rad_sc']+edges_dual['land_sc']

In [ ]:
nodes_dual.gdf_name = 'Dual_list'
Gr = ox.gdfs_to_graph(nodes_dual, edges_dual)
    
n = Gr.nodes()
pos = {}
    
print(len(n))
for l, item in enumerate(n): pos[l] = (n[l]['x'],n[l]['y'],n[l]['streetID'])
        
DG = nx.Graph() #Empty graph
DG = DG.to_undirected()
DG.add_nodes_from(pos.keys()) #Add nodes preserving coordinates
    
for i, item in enumerate(DG.nodes()):
    DG.node[i]['x']=pos[i][0]
    DG.node[i]['y']=pos[i][1]
    DG.node[i]['streetID']=pos[i][2]
        
for i, item in enumerate(Gr.edges()):
    DG.add_edge(item[0], item[1])
    DG[item[0]][item[1]]['angle&land'] = Gr[item[0]][item[1]][0]['angle&land']

In [ ]:
ALb = nx.betweenness_centrality(DG, weight='angle&land', normalized=False)

In [ ]:
def id_dict(ed, graph):
    
    view = ed.items()
    ed_list = list(view)
    ed_dict = {}

    for p in ed_list:
        ed_dict[graph.node[p[0]]['streetID']]=p[1] #streetID and Edge betweenness
        
    return(ed_dict)

In [ ]:
ALb_dict = id_dict(ALb, DG)
ALb_df = to_df([ALb_dict], ["ALb"])

In [ ]:
paths_tmp = pd.merge(paths_tmp, ALb_df, left_on="streetID", right_index=True, how='left')

In [ ]:
plot(paths_tmp,'Ab', 'Angular Betweenness')
plot(paths_tmp,'ALb', 'Angular-Landmark Betweenness')

In [ ]:
paths_tmp.to_file(folder_ouptut+city_name+'_paths.shp', driver='ESRI Shapefile')

In [ ]:
nodes.to_file(folder_ouptut+city_name+'_nodes.shp', driver='ESRI Shapefile')

In [ ]:
paths_tmp.head()

In [ ]:
pedestrians = pd.read_csv("C:/Users/g_filo01/sciebo/GIS Data/Simulation/pedestrians_eucl.csv")

In [ ]:
pedestrians.dtypes

In [ ]:
colPed = ["euclidean", "topological", "angular", "landmark", "euclideanLand", "topologicalLand", "angularLand"]

In [ ]:
paths_data = pd.merge(paths_tmp, pedestrians, left_on="streetID", right_on="streetID", how='left')

In [ ]:
paths_data['euclideanDiff']=paths_data['euclidean']-paths_data['euclideanLand']

In [ ]:
len(paths_data)

In [ ]:
plotGDF(paths_data,"euclidean","euclidean")
# plotGDF(paths_data,"topological","topological")
# plotGDF(paths_data,"angular","angular")
plotGDF(paths_data,"euclideanLand","euclideanLand")
plotGDF(paths_data,"euclideanDiff","diff")
# plotGDF(paths_data,"topologicalLand","topologicalLand")
# plotGDF(paths_data,"angularLand","angularLand")
# plotGDF(paths_data,"landmark","landmark")

In [ ]:
for i in colPed:
    print(i)
    plotGDF(paths_data, colPed, colPed)